In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "mumbai_india.osm"  # Replace this with your osm file
SAMPLE_FILE = "mumbai_sample.osm"

k = 500 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')


In [1]:
import xml.etree.cElementTree as ET
import pprint
tags={}
def count_tags(sample_file):
    for event, elem in ET.iterparse(sample_file, events=("start",)):
        if elem.tag in tags.keys():
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
        #print tags
    return tags
tags = count_tags('mumbai_india.osm')
pprint.pprint(tags)

{'bounds': 1,
 'member': 13085,
 'nd': 2356083,
 'node': 2051208,
 'osm': 1,
 'relation': 3994,
 'tag': 391731,
 'way': 283963}


In [12]:
import xml.etree.cElementTree as ET
import pprint
import re
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
            if lower.search(element.attrib['k']):
              keys["lower"] += 1
            elif lower_colon.search(element.attrib['k']):
              keys["lower_colon"] += 1
            elif problemchars.search(element.attrib['k']):
              keys["problemchars"] += 1
            else:
              keys["other"] += 1
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys
keys = process_map('mumbai_sample.osm')
pprint.pprint(keys)

{'lower': 761, 'lower_colon': 52, 'other': 1, 'problemchars': 0}


In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
def get_user(element):
     if "uid" in element.attrib:
        return element.attrib["uid"]


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
       users.add(get_user(element))
       users.discard(None)
    return users
users = process_map('mumbai_sample.osm')
#pprint.pprint(users)
len(users)


213

In [3]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "mumbai_india.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#expected = ["Slums", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            
# UPDATE THIS VARIABLE
mapping = { "Jhopadpatti": "Slums",
            "Marg,": "Marg",
            "Mumbai,":"Mumbai",
            "ROad":"Road",
            "Rd":"Road",
            "Raod":"Road",
            "Rd.":"Road",
            "road":"Road"
            }




# UPDATE THIS VARIABLE


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        #if street_type not in expected:
        street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    #print '\nSTART'
    #print name
    m = street_type_re.search(name)
    if m.group() in mapping.keys():        
        name=re.sub(street_type_re,mapping[m.group()],name)

    # YOUR CODE HERE

    return name

st_types = audit(OSMFILE)
#pprint.pprint(dict(st_types))

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name  

Universal Mobile,DVD and Travelling shop => Universal Mobile,DVD and Travelling shop
Vidhan Bhavan => Vidhan Bhavan
kurla => kurla
lalluabai compound => lalluabai compound
THAKKAR BAPPA COLONY => THAKKAR BAPPA COLONY
DATAR COLONY => DATAR COLONY
P.L. Lokhande Marg,MHADA COLONY => P.L. Lokhande Marg,MHADA COLONY
Thakurli Station Rd => Thakurli Station Road
Veer Savarkar Rd => Veer Savarkar Road
Bhandup Village Rd => Bhandup Village Road
Sambhaji Nagar Rd => Sambhaji Nagar Road
Pipeline Rd => Pipeline Road
Vasai Station Rd => Vasai Station Road
NL Rd => NL Road
Marol Village Rd => Marol Village Road
Vishnu Nagar Rd => Vishnu Nagar Road
Yashavant Nagar Rd => Yashavant Nagar Road
vishnu Nagar Rd => vishnu Nagar Road
Balraj Sahni Rd => Balraj Sahni Road
Wodehouse Rd => Wodehouse Road
Koparkhairane Village Rd => Koparkhairane Village Road
Datta Mandir Rd => Datta Mandir Road
Belpada Rd => Belpada Road
Goregaon Mulund Link Rd => Goregaon Mulund Link Road
Agrawal Rd => Agrawal Road
Carmichael 

Dr. Dadabhai Naoroji => Dr. Dadabhai Naoroji
l.j.road => l.j.road
Sector 6 Market => Sector 6 Market
malad west => malad west
Sumer castle => Sumer castle
M.G.Road => M.G.Road
Ramchandra Gali => Ramchandra Gali
new link road => new link Road
st anthony church  road => st anthony church  Road
mohd ali road => mohd ali Road
ghatkapar/mankhurd link road => ghatkapar/mankhurd link Road
jay bhavani mata road => jay bhavani mata Road
LBS Marg 90 feet road => LBS Marg 90 feet Road
2nd floor swadesi market kalbadevi road => 2nd floor swadesi market kalbadevi Road
saba road => saba Road
video road => video Road
Shankar Pawse road => Shankar Pawse Road
devidas ext road => devidas ext Road
tanki road => tanki Road
father peter periera road => father peter periera Road
24th road => 24th Road
Godrej Hill road => Godrej Hill Road
4th road => 4th Road
Gograswadi, Patharli road => Gograswadi, Patharli Road
Kamothe main road => Kamothe main Road
Barave gaon road => Barave gaon Road
Sher-e-Punjab road =

In [5]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "mumbai_india.osm"
tree = ET.parse(OSMFILE)
root = tree.getroot()

postal_type_re = re.compile('^(4)(0)\d{4}$')




expected = [ 'tiger:zip_left','tiger:zip_left_1','tiger:zip_left_2',
             'tiger:zip_left_3','tiger:zip_left_4', 'tiger:zip_right',
             'tiger:zip_right_1','tiger:zip_right_2','tiger:zip_right_3',
             'tiger:zip_right_4','addr:postcode']

def audit_postal_code(postal_types,postal_value):
    m = postal_type_re.search(postal_value)
    if m:
        postal_type=m.group()
        postal_types[postal_type].add(postal_value)

def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    postal_types=defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag in["node", "way", "relation"] :
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(postal_types,tag.attrib['v'])
    osm_file.close()
    return postal_types


#audit(SAMPLE_FILE)
#audit(OSMFILE)
#pprint.pprint(dict(audit(OSMFILE)))



In [5]:
import xml.etree.ElementTree as ET
import re

tree = ET.parse(OSMFILE)
root = tree.getroot()

counttotal = 0
count = 0
wp = []
regex = re.compile('^(4)(0)\d{4}$')
for i in tree.getiterator('tag'):
    k1 = i.get("k")
    if k1 == "addr:postcode":
        v1 = i.get("v")
        m1 = regex.match(v1)
        if not m1:
            counttotal = counttotal +1
            if len(v1) <> 6:
                v1 = v1.replace(" ","")
                v1 = v1.replace(",","")
                v1 = v1.replace("-","")
                v1 = v1.replace('"',"")
                v1 = v1.replace('55',"5")
                m2 = regex.match(v1)
                if not m2:
                    wp.append(v1)
                    count = count +1
            elif len(v1) == 6:
                wp.append(v1)
                count = count +1

print wp
print count
#print counttotal


['421501', '421503', '410206', '410210', '410210', '410210', '410210', '412108', '410701', '421202', '421202', '4000607', '40058', '421501', '63103', '421501', '421501', '410201', '410201', '410201', '4000082', '400059.', '410210', '110092', '123', '410209', '410209', '410209', '421302', '410206', '4000072', '410206', '421301', '421301', '4000072', '421306', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410210', '410208', '421501', '410206', '40076', '560023', '590006', '492006', '410210', '410206', '560092', '410206', '410206', '410206', '410206', '410206', '410206', '410206', '410206', '410206', '410221', '410102', '410206', '421601', '410206', '421005', '410206', '421501', '410221', '410206', '410206', '48147', '410218', '421201', '421302', '410206', '412108', '412108', '421501', '421501', '410206', '421311', '400076India', '400076India', '410210', '40081', '4

In [6]:
def update_postcode(postcode):
    search = re.search(r'^\D*(\d{6}).*',postcode)
    search1 = re.search('^(4)(0)\d{4}$',postcode)
    if search:
        return search.group(1)
    elif search1:
        return search1.group(1)
    else :
        return '000000'
for i in tree.getiterator('tag'):
    k1 = i.get("k")
    if k1 == "addr:postcode":
        v1 = i.get("v")
        print  update_postcode(v1)
        

421501
421503
410206
400050
400050
400050
400050
400615
400614
410210
400005
410210
400020
400086
400601
400022
400606
400039
400005
400001
400039
400020
000000
400039
410210
400039
400607
400020
400069
400703
400610
400016
400071
400067
400005
400049
400072
400710
400093
400054
400052
400088
410210
400001
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400057
400607
400607
400607
400607
400607
400607
400607
400607
400607

400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400005
400038
400093
400062
410210
410210
410210
410210
410210
410210
410210
410208
410208
410209
410210
400086
400708
400001
400050
000000
000000
400053
400061
400061
400058
400028
400082
400043
400001
400051
400005
400020
400005
400018
400013
400001
400001
400088
400005
400708
400709
400705
400701
410206
400089
400071
400081
400081
400081
400081
400081
400005
400081
400080
000000
000000
400086
400070
400001
400071
400050
400050
400050
400050
400050
400050
400050
400050
400051
400051
400051
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400080
400050
400050
400050
400050
400050
400050
400050
400050
400080
400081
400050
400050
400050
400050
412108
000000
400070
400010
400079
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050
400050

In [ ]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "mumbai_india.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
  
    # YOUR CODE HERE
    
    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            tag_dict= {}
            # Calling the cleaning function
            # replace name with the value of the attribute
            if tag.attrib['k'] == 'addr:street':
                tag.attrib['v'] = update_name(tag.attrib['v'], mapping)
            if tag.attrib["k"] == 'addr:postcode':
                tag.attrib["v"] = update_postcode(tag.attrib["v"])
            
            tag_dict['id'] = node_attribs['id']
            key = tag.attrib['k']
            
            if re.search(PROBLEMCHARS, tag.attrib['k']):
                pass
            if re.search(LOWER_COLON, tag.attrib['k']):
                pass
            if ':' in tag.attrib['k']:
                type = key[: key.index(':')]
                key = key[key.index(':')+1 :]   
            else:
                type = 'regular'   
            
            tag_dict['key'] = key
            tag_dict['value'] = tag.attrib['v']
            tag_dict['type'] = type
            tags.append(tag_dict)
            
            #pprint.pprint(dict(tag_dict))
            
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]
        for tag in element.iter("tag"):
            tag_dict1= {}
            if tag.attrib['k'] == 'addr:street':
                tag.attrib['v'] = update_name(tag.attrib['v'], mapping)
            if tag.attrib["k"] == 'addr:postcode':
                tag.attrib["v"] = update_postcode(tag.attrib["v"])
            tag_dict1['id'] = way_attribs['id']
            key = tag.attrib['k']
            
            if re.search(PROBLEMCHARS, tag.attrib['k']):
                pass
            if re.search(LOWER_COLON, tag.attrib['k']):
                pass
            if ':' in tag.attrib['k']:
                type = key[: key.index(':')]
                key = key[key.index(':')+1 :]
            else:
                type = 'regular' 
            tag_dict1['key'] = key
            tag_dict1['value'] = tag.attrib['v']
            tag_dict1['type'] = type
            tags.append(tag_dict1) 
            
        i= 0
        for tag in element.iter("nd"):
            way_dict = {}
            way_dict["id"] = way_attribs["id"]
            way_dict["node_id"] = tag.attrib["ref"]
            way_dict["position"] = i
            way_nodes.append(way_dict)
            i +=1    
        
        # pprint.pprint(dict(way_dict)) 
            
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'wb') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


In [20]:
amenity_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def audit_amenity_type(amenity_types, amenity_name):
    m = amenity_type_re.search(amenity_name)
    if m:
        amenity_type = m.group()
        #if street_type not in expected:
        amenity_types[amenity_type].add(amenity_name)


def is_amenity_name(elem):
    return (elem.attrib['k'] == "amenity")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    amenity_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_amenity_name(tag):
                    audit_amenity_type(amenity_types, tag.attrib['v'])
    osm_file.close()
    return amenity_types

def update_name(name, mapping):
    #print '\nSTART'
    #print name
    m = street_type_re.search(name)
    if m.group() in mapping.keys():        
        name=re.sub(street_type_re,mapping[m.group()],name)

    # YOUR CODE HERE

    return name

st_types = audit(OSMFILE)
#pprint.pprint(dict(st_types))

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name  

taxi => taxi
shop => shop
social_facility => social_facility
clinic => clinic
embassy => embassy
post_office => post_office
bureau_de_change => bureau_de_change
toilets => toilets
police => police
townhall => townhall
community_centre => community_centre
internet_cafe => internet_cafe
pharmacy => pharmacy
kindergarten => kindergarten
prison => prison
fuel => fuel
waste_basket => waste_basket
meditation_centre => meditation_centre
bank => bank
place_of_worship => place_of_worship
school => school
bar => bar
parking_entrance => parking_entrance
university => university
crematorium => crematorium
fairgrounds => fairgrounds
grave_yard => grave_yard
library => library
college => college
parking => parking
canteen => canteen
yes => yes
check_in => check_in
food_court => food_court
cyber_cafe => cyber_cafe
veterinary => veterinary
cold_storage => cold_storage
conference_centre => conference_centre
fountain => fountain
public => public
fast_food => fast_food
post_box => post_box
pub => pub
wat

In [21]:
amenity_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def audit_amenity_type(amenity_types, amenity_name):
    m = amenity_type_re.search(amenity_name)
    if m:
        amenity_type = m.group()
        #if street_type not in expected:
        amenity_types[amenity_type].add(amenity_name)


def is_amenity_name(elem):
    return (elem.attrib['k'] == "phone")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    amenity_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_amenity_name(tag):
                    audit_amenity_type(amenity_types, tag.attrib['v'])
    osm_file.close()
    return amenity_types

st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))

{'0000': set(['+91 22 2838 0000']),
 '0004': set(['+91 97 6936 0004']),
 '0195': set(['+91 22 2265 0195']),
 '0201': set(['+91 22 2745 0201']),
 '02030189817': set(['02030189817']),
 '0212': set(['6519 0212']),
 '022-25336112': set(['022-25336112']),
 '022-25565041/25565051': set(['022-25565041/25565051']),
 '022-28455353': set(['022-28455353']),
 '022-28575099': set(['022-28575099']),
 '022-3988-3988': set(['022-3988-3988']),
 '02222042897': set(['02222042897']),
 '02223453372': set(['02223453372']),
 '02223876457': set(['02223876457']),
 '02224102623': set(['02224102623']),
 '02224306415': set(['+91 02224306415']),
 '02225390506': set(['02225390506']),
 '02225974747': set(['02225974747']),
 '02226206021': set(['02226206021']),
 '02226744356': set(['02226744356']),
 '02227552019': set(['02227552019']),
 '02227571324': set(['02227571324']),
 '02227572111': set(['02227572111']),
 '02227572293': set(['02227572293']),
 '02227580255': set(['02227580255']),
 '02227700376': set(['02227700376

In [2]:
import csv, sqlite3
from pprint import pprint
sql_file="Mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()

cur.execute('''DROP TABLE IF EXISTS nodes;''')
con.commit()


cur.execute("CREATE TABLE nodes(id INTEGER, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp DATE);") # use your column names here
con.commit()
with open('nodes.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
con.commit()
#con.close()


cur.execute('''DROP TABLE IF EXISTS nodes_tags;''')
con.commit()

cur.execute("CREATE TABLE Nodes_tags (id INTEGER, key TEXT, value TEXT, type TEXT);") # use your column names here
con.commit()
with open('nodes_tags.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO Nodes_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
con.commit()

cur.execute('''DROP TABLE IF EXISTS ways;''')
con.commit()


cur.execute("CREATE TABLE ways (id INTEGER, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp DATE);") # use your column names here
con.commit()
with open('ways.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO ways (id , user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
con.commit()


cur.execute('''DROP TABLE IF EXISTS ways_tags;''')
con.commit()


cur.execute("CREATE TABLE ways_tags (id INTEGER, key TEXT, value TEXT, type TEXT);") # use your column names here
con.commit()
with open('ways_tags.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]


cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
con.commit()



cur.execute('''DROP TABLE IF EXISTS ways_nodes;''')
con.commit()

cur.execute("CREATE TABLE ways_nodes (id INTEGER, node_id INTEGER, position INTEGER);") # use your column names here
con.commit()
with open('ways_nodes.csv','rb') as thr: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(thr) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]
SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM node_tags UNION ALL 
      SELECT * FROM way_tags) tags
WHERE tags.key LIKE '%city'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 10;
cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
con.commit()


In [36]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT user,count(*) FROM nodes GROUP BY user ORDER BY count(*) DESC LIMIT 5')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'parambyte', 69327),
 (u'PlaneMad', 68363),
 (u'anushap', 62553),
 (u'Ashok09', 62208),
 (u'Narsimulu', 55611)]


In [2]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT key,count(*) FROM ways_tags GROUP BY key ORDER BY count(*) DESC LIMIT 5')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

NameError: name 'sqlite3' is not defined

In [25]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(1739,)]


In [ ]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT COUNT(*) FROM (SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) as e GROUP BY e.user HAVING num<=5)  u')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

In [ ]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="restaurant") as i ON nodes_tags.id=i.id WHERE nodes_tags.key="cuisine" GROUP BY nodes_tags.value ORDER BY num DESC')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

In [28]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags  JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="place_of_worship") as i ON nodes_tags.id=i.id WHERE nodes_tags.key="religion" GROUP BY nodes_tags.value ORDER BY num DESC')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'hindu', 125),
 (u'muslim', 71),
 (u'christian', 34),
 (u'buddhist', 13),
 (u'jain', 6),
 (u'sikh', 4),
 (u'zoroastrian', 2),
 (u'jewish', 1)]


In [29]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT nodes_tags.value, count(*) as num FROM nodes_tags  WHERE nodes_tags.key="leisure" GROUP BY nodes_tags.value ORDER BY num DESC')
all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'park', 63),
 (u'playground', 20),
 (u'sports_centre', 15),
 (u'garden', 10),
 (u'fitness_centre', 8),
 (u'pitch', 6),
 (u'swimming_pool', 4),
 (u'aquarium', 1),
 (u'fitness_station', 1),
 (u'golf_course', 1),
 (u'stadium', 1)]


In [3]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT value, count(*) as num FROM nodes_tags WHERE key="street" GROUP BY value ORDER BY num DESC LIMIT 10')

all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'Hanuman Road', 78),
 (u'Yashavant Nagar Road', 29),
 (u'Hiranandani Estate', 24),
 (u'P.L. Lokhande Marg', 24),
 (u'New Link Road, Andheri West', 21),
 (u'LBS Marg', 18),
 (u'Road Number 3', 18),
 (u'Thane Ghodbunder Road', 18),
 (u'Eastern Express Highway', 14),
 (u'GD Somani Road', 13)]


In [6]:
sql_file="mumbai_india.db"
con = sqlite3.connect(sql_file)
cur = con.cursor()
cur.execute('SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL  SELECT * FROM ways_tags) tags WHERE tags.key LIKE "%city" GROUP BY tags.value ORDER BY count DESC LIMIT 10')

all_rows=cur.fetchall()
pprint(all_rows)


con.close()

[(u'Mumbai', 607),
 (u'Bandra, Mumbai', 566),
 (u'mumbai', 187),
 (u'Virar West', 91),
 (u'Mulund (West)', 79),
 (u'Navi Mumbai', 70),
 (u'MUMBAI', 68),
 (u'Mulund (East)', 62),
 (u'Thane', 49),
 (u'Kharghar', 43)]
